In [1]:
import pyodbc
import os

In [2]:
os.environ['USERNAME'] = 'avish'
os.environ['PASSWORD'] = 'avish'

In [ ]:
USER = os.environ['USERNAME']

In [ ]:
USER

In [ ]:
PASSWORD = os.environ['PASSWORD']

In [ ]:
PASSWORD

### Connect to the database

In [4]:
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=AVISHEK\SQLEXPRESS;"
                      "Database=ecommerce;"
                      "Trusted_Connection=yes;")

In [ ]:
cursor = conn.cursor()
# let's execute a query to check the connection
cursor.execute('SELECT * FROM customers')

for row in cursor:
    print('row = %r' % (row,))

In [ ]:
# delete cursor object
del cursor

In [ ]:
cursor

In [9]:
def getProductsByType(db_conn, prod_type):
    cursor = db_conn.cursor()
    query = "SELECT * FROM products WHERE product_type = ?"
    cursor.execute(query, prod_type)
    return cursor.fetchall()

In [10]:
all_ovens = getProductsByType(conn, 'oven')

In [11]:
all_ovens

[('1', 'Reflector oven', 'oven'),
 ('2', 'Convection microwave', 'oven'),
 ('15', 'Masonry oven', 'oven'),
 ('24', 'Microwave oven', 'oven'),
 ('32', 'Russian oven', 'oven'),
 ('33', 'Clome oven', 'oven'),
 ('34', 'Convection oven', 'oven'),
 ('37', 'Beehive oven', 'oven'),
 ('38', 'Toaster and toaster ovens', 'oven'),
 ('41', 'Self-cleaning oven', 'oven'),
 ('42', 'Wood-fired oven', 'oven'),
 ('43', 'Kitchener range', 'oven'),
 ('47', 'Oven', 'oven'),
 ('48', 'Hot box (appliance)', 'oven'),
 ('58', 'Tabun oven', 'oven'),
 ('67', 'Chorkor oven', 'oven'),
 ('72', 'Communal oven', 'oven'),
 ('74', 'Halogen oven', 'oven'),
 ('78', 'Stove', 'oven'),
 ('80', 'Earth oven', 'oven')]

In [14]:
cursor = conn.cursor()
fetchall_cnt = 0

for oven in all_ovens:
    oven_id = oven[0]
    query = "SELECT * FROM product_orders WHERE product_id = ?"
    cursor.execute(query, oven_id)
    oven_order = cursor.fetchall()
    fetchall_cnt += 1
    print(oven_order, "\n")
    print("fetchall_cnt:", fetchall_cnt, "\n")

[(3, 1, 5, Decimal('72.69')), (25, 1, 4, Decimal('63.59')), (59, 1, 3, Decimal('22.07')), (68, 1, 4, Decimal('196.11')), (71, 1, 5, Decimal('227.35')), (81, 1, 4, Decimal('116.89')), (121, 1, 2, Decimal('277.29')), (123, 1, 1, Decimal('127.55')), (138, 1, 1, Decimal('38.29')), (155, 1, 1, Decimal('279.34')), (187, 1, 3, Decimal('149.28')), (203, 1, 1, Decimal('299.98')), (208, 1, 1, Decimal('121.73')), (259, 1, 1, Decimal('167.60')), (300, 1, 1, Decimal('66.57'))] 

fetchall_cnt: 1 

[(20, 2, 1, Decimal('211.07')), (57, 2, 4, Decimal('65.81')), (58, 2, 5, Decimal('102.97')), (63, 2, 4, Decimal('43.22')), (79, 2, 2, Decimal('293.17')), (80, 2, 1, Decimal('181.54')), (94, 2, 4, Decimal('297.49')), (130, 2, 3, Decimal('44.08')), (141, 2, 3, Decimal('187.80')), (155, 2, 2, Decimal('24.27')), (185, 2, 3, Decimal('246.71')), (237, 2, 3, Decimal('224.47')), (253, 2, 4, Decimal('298.50')), (263, 2, 2, Decimal('112.06')), (273, 2, 2, Decimal('182.99'))] 

fetchall_cnt: 2 

[(10, 15, 5, Decimal(

And what you'll notice is, each time that fetchall count is being incremented, that's indicating we're doing another fetch to the database. And that's kind of like going to the store, and picking up one item, going home, dropping it off, going back to the store, pick up your next item, go home, drop it off and so on. A much better way to do it, whether you're fetching data from a database or getting groceries from the store, is to get all of the items you need at one time, in one trip, rather than going back and forth. So let's take a look at how to do that. 

In [15]:
def getProductOrdersByType(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = "SELECT po.*   \
                    FROM product_orders po  \
                    INNER JOIN products p   \
                    ON po.product_id = p.product_id  \
                    WHERE p.product_type = ?"
    cursor.execute(sql_string, prod_type)
    return(cursor.fetchall())

In [16]:
getProductOrdersByType(conn, 'oven')

[(3, 1, 5, Decimal('72.69')),
 (4, 74, 5, Decimal('147.32')),
 (6, 58, 2, Decimal('110.55')),
 (7, 42, 5, Decimal('201.44')),
 (9, 43, 5, Decimal('230.13')),
 (9, 80, 5, Decimal('111.15')),
 (10, 15, 5, Decimal('76.80')),
 (11, 15, 1, Decimal('282.80')),
 (13, 42, 1, Decimal('47.79')),
 (14, 58, 4, Decimal('198.79')),
 (15, 38, 2, Decimal('299.62')),
 (15, 41, 2, Decimal('158.61')),
 (17, 41, 2, Decimal('114.64')),
 (18, 41, 2, Decimal('126.41')),
 (20, 2, 1, Decimal('211.07')),
 (21, 47, 3, Decimal('73.83')),
 (23, 32, 5, Decimal('114.08')),
 (24, 32, 5, Decimal('60.54')),
 (25, 1, 4, Decimal('63.59')),
 (26, 43, 2, Decimal('195.19')),
 (27, 67, 2, Decimal('191.86')),
 (27, 78, 3, Decimal('119.16')),
 (28, 43, 5, Decimal('296.70')),
 (29, 74, 1, Decimal('252.93')),
 (29, 80, 3, Decimal('123.98')),
 (30, 32, 1, Decimal('57.09')),
 (30, 34, 3, Decimal('61.61')),
 (32, 24, 3, Decimal('70.57')),
 (33, 38, 4, Decimal('236.11')),
 (35, 32, 1, Decimal('148.47')),
 (35, 47, 5, Decimal('261.38

So in general, anytime you're working with multiple tables, you want to avoid and plus one queries. So don't do one query, get a list stored in memory, and then iterate over that. That's really inefficient. Relational databases today, are really well optimized, for doing the joint operation. And they can do it much more efficiently, in the database engine, than we can do in our application.

### Challange

In [24]:
def getProductOrdersByTypeQuantity(db_conn, prod_type, quantity):
    cursor = db_conn.cursor()
    sql_string = "SELECT po.*   \
                    FROM product_orders po  \
                    INNER JOIN products p   \
                    ON po.product_id = p.product_id  \
                    WHERE p.product_type = ? \
                    AND po.quantity > ?"
    cursor.execute(sql_string, (prod_type, quantity))
    return(cursor.fetchall())

In [25]:
getProductOrdersByTypeQuantity(conn, 'fryer', 3)

[(7, 64, 5, Decimal('261.31')),
 (10, 3, 4, Decimal('90.93')),
 (24, 3, 5, Decimal('131.20')),
 (56, 17, 5, Decimal('64.47')),
 (75, 54, 4, Decimal('165.38')),
 (86, 3, 5, Decimal('266.13')),
 (90, 64, 5, Decimal('133.20')),
 (98, 54, 4, Decimal('45.03')),
 (140, 3, 5, Decimal('298.42')),
 (150, 3, 5, Decimal('259.76')),
 (150, 54, 5, Decimal('62.33')),
 (151, 3, 4, Decimal('262.93')),
 (183, 64, 5, Decimal('58.37')),
 (205, 64, 5, Decimal('228.90')),
 (220, 17, 5, Decimal('146.25')),
 (224, 17, 4, Decimal('30.91')),
 (254, 26, 4, Decimal('22.24')),
 (273, 3, 4, Decimal('222.86')),
 (295, 17, 5, Decimal('143.12'))]